# Topic Modelling
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [19]:
import json
from pathlib import Path
import glob
from nltk.corpus import stopwords
from dateutil import parser

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [14]:
stop_words = stopwords.words('german')

## Load Data

In [15]:
records = []
for f in glob.glob(str(Path('../export/tags/*.json'))):
    records = records + json.load(open(f, 'r', encoding='utf-8'))

print(len(records))

75264


In [16]:
# Members only, no Presidents
r_members = list(filter(lambda x: x['ismember'] == True, records))
r_members = list(filter(lambda x: x['funktion'] not in ['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'], r_members))
print(len(r_members))

55623


In [ ]:
## Filter by date if needed

In [33]:
from_date = parser.parse("2011-05-01")
r_members = list(filter(lambda x: parser.parse(x['sitzung_date']) >= from_date, r_members))
print(len(r_members))

31321

## Prepare Data

In [34]:
# Remove Stopwords and only keep noun, adj, vb and adv

keep = ['NN', 'NE',
    'ADJA', 'ADJD',
    'VMFIN', 'VAFIN', 'VVFIN', 'VAIMP', 'VVIMP', 'VVINF', 'VAINF', 'VMINF', 'VVIZU', 'VVPP', 'VMPP', 'VAPP',
    'ADV']

for votum in r_members:
    # Reversed Loop, we will remove items from the list during loop!
    for i in reversed(range(0, len(votum['tags']))):
        tag = votum['tags'][i]
        if tag[0].lower() in stop_words:

            # Stopword found. Remove
            votum['tags'].remove(tag)

        elif tag[2] not in keep:
            
            # Not tag we need. Remove
            votum['tags'].remove(tag)

In [35]:
# Stick the words together to get a nice list for our corpora and split by sex
data_m = []
data_w = []
for votum in r_members:
    if votum['geschlecht'] == 'm':
        data_m.append(list(map(lambda tag: tag[0], votum['tags'])))
    else:
        data_w.append(list(map(lambda tag: tag[0], votum['tags'])))

# Build Dictionary
id2word_m = corpora.Dictionary(data_m)
id2word_w = corpora.Dictionary(data_w)

# Term Document Frequency
corpus_m = [id2word_m.doc2bow(text) for text in data_m]
corpus_w = [id2word_w.doc2bow(text) for text in data_w]

## Go LDA

In [36]:
lda_model_m = gensim.models.ldamodel.LdaModel(corpus=corpus_m, id2word=id2word_m, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [37]:
lda_model_w = gensim.models.ldamodel.LdaModel(corpus=corpus_w, id2word=id2word_w, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [53]:
root = Path('../export/models/2011-05-01/')

# Store LDA
lda_model_m.save(str(root / Path('lda_m.model')))
lda_model_w.save(str(root / Path('lda_w.model')))

# Store corpus & id2word
with open(root / Path('corpus_m'), 'w', encoding='utf-8') as f:
    f.write(json.dumps(corpus_m, ensure_ascii = False))
id2word_m.save(str(root / Path('id2word_m')))

with open(root / Path('corpus_w'), 'w', encoding='utf-8') as f:
    f.write(json.dumps(corpus_w, ensure_ascii = False))
id2word_w.save(str(root / Path('id2word_w')))

## Load models, if needed

In [66]:
root = Path('../export/models/2011-05-01/')

In [7]:
# If needed...
lda_model_m = gensim.models.LdaModel.load(str(root / Path('lda_m.model')))
lda_model_w = gensim.models.LdaModel.load(str(root / Path('lda_w.model')))

In [65]:
id2word_m = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_m'))
id2word_w = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_w'))

corpus_m = json.loads(open(root / Path('corpus_m'), 'r', encoding='utf-8').read())
corpus_w = json.loads(open(root / Path('corpus_w'), 'r', encoding='utf-8').read())

## Men

In [8]:
pyLDAvis.enable_notebook()

In [9]:
lda_viz = gensimvis.prepare(lda_model_m, corpus_m, id2word_m)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module i

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.407514 -0.239036       1        1  43.410606
7      0.399823 -0.183249       2        1  29.588215
11     0.233329  0.269489       3        1   7.170819
0      0.227852  0.164049       4        1   5.677704
6      0.162130  0.121625       5        1   3.885961
16     0.106199  0.137753       6        1   3.299881
5      0.004598  0.022606       7        1   1.727705
4     -0.077372 -0.017569       8        1   0.974141
13    -0.069092 -0.012524       9        1   0.820206
8     -0.100863 -0.020950      10        1   0.611438
12    -0.103073 -0.021533      11        1   0.506021
9     -0.097146 -0.020152      12        1   0.477176
17    -0.120473 -0.024217      13        1   0.429802
19    -0.120606 -0.024317      14        1   0.309297
14    -0.134718 -0.025472      15        1   0.255448
2     -0.137709 -0.024520      16        1   0.255273
18    -0.141685 -0.025635      17        1   0.200705
3     -0.143354 -0.025621      18        1   0.180549
10    -0.142924 -0.025627      19        1   0.176935
15    -0.152431 -0.025100      20        1   0.042116, topic_info=                Term          Freq         Total Category  logprob  loglift
50            Zürich  50901.000000  50901.000000  Default  30.0000   30.000
82            Kanton  50336.000000  50336.000000  Default  29.0000   29.000
176        Gemeinden  16430.000000  16430.000000  Default  28.0000   28.000
174          Franken  25997.000000  25997.000000  Default  27.0000   27.000
133           Antrag  21547.000000  21547.000000  Default  26.0000   26.000
...              ...           ...           ...      ...      ...      ...
452             Lü16      0.008934      1.203921  Topic20 -12.3698    2.869
4172   einverstanden      0.008934      1.203914  Topic20 -12.3698    2.869
6704        falschen      0.008934      1.203895  Topic20 -12.3698    2.869
10547         Tunnel      0.008934      1.203922  Topic20 -12.3698    2.869
4768         grosser      0.008934      1.203926  Topic20 -12.3698    2.869

[749 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
128       2  0.552448            1.
128       5  0.160691            1.
128      11  0.286780            1.
1319     12  0.983596     Abgesehen
2214      5  0.999672        Absatz
...     ...       ...           ...
4377     15  0.993584   überfordert
4256      8  0.961917  überlasteten
3254     19  0.989276    übermässig
3896     20  0.985137    überprüfen
2079     12  0.979279   überraschen

[597 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 12, 1, 7, 17, 6, 5, 14, 9, 13, 10, 18, 20, 15, 3, 19, 4, 11, 16])

## Women

In [10]:
lda_viz = gensimvis.prepare(lda_model_w, corpus_w, id2word_w)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.426798 -0.223716       1        1  40.301086
17    -0.430969 -0.088661       2        1  36.530339
2     -0.243396  0.276162       3        1   6.239699
8     -0.150269  0.240755       4        1   5.151001
15    -0.048364  0.047281       5        1   2.820128
19    -0.032207  0.027643       6        1   2.495486
12     0.035994 -0.016677       7        1   1.520561
16     0.027338 -0.014013       8        1   1.398550
1      0.063841 -0.022544       9        1   0.904952
11     0.080849 -0.019844      10        1   0.696120
3      0.099297 -0.024244      11        1   0.503403
9      0.107122 -0.023172      12        1   0.423481
5      0.108758 -0.022763      13        1   0.250720
18     0.114125 -0.020698      14        1   0.191428
14     0.114811 -0.020265      15        1   0.164466
0      0.115324 -0.019888      16        1   0.141640
4      0.114411 -0.020527      17        1   0.129666
6      0.116052 -0.019236      18        1   0.079711
7      0.116434 -0.018833      19        1   0.050787
13     0.117648 -0.016759      20        1   0.006775, topic_info=                Term          Freq         Total Category  logprob  loglift
150        Gemeinden   8091.000000   8091.000000  Default  30.0000  30.0000
882         Postulat   6895.000000   6895.000000  Default  29.0000  29.0000
66            Zürich  21957.000000  21957.000000  Default  28.0000  28.0000
2             Antrag   7236.000000   7236.000000  Default  27.0000  27.0000
20            Kanton  20251.000000  20251.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
6628         oberste      0.001069      1.018302  Topic20 -11.7854   2.7402
7059  vorgeschrieben      0.001069      1.018308  Topic20 -11.7854   2.7402
7266            Erst      0.001069      1.018316  Topic20 -11.7854   2.7401
7405        schlicht      0.001069      1.018345  Topic20 -11.7854   2.7401
8217           komme      0.001069      1.018336  Topic20 -11.7854   2.7401

[812 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3383     11  0.982570           28.
3186     11  0.998034           ABG
228       6  0.998250            AL
2885     12  0.994235   Abklärungen
0         3  0.998854     Ablehnung
...     ...       ...           ...
604       6  0.998523   Überweisung
4718      2  0.999794  öffentlichen
3253     11  0.995884       überall
4492      7  0.996516    übertragen
1514      6  0.997353    überwiesen

[567 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 18, 3, 9, 16, 20, 13, 17, 2, 12, 4, 10, 6, 19, 15, 1, 5, 7, 8, 14])

## 10 Years back

In [67]:
root = Path('../export/models/2011-05-01/')

lda_model_m = gensim.models.LdaModel.load(str(root / Path('lda_m.model')))
lda_model_w = gensim.models.LdaModel.load(str(root / Path('lda_w.model')))

id2word_m = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_m'))
id2word_w = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_w'))

corpus_m = json.loads(open(root / Path('corpus_m'), 'r', encoding='utf-8').read())
corpus_w = json.loads(open(root / Path('corpus_w'), 'r', encoding='utf-8').read())

### Men

In [69]:
lda_viz = gensimvis.prepare(lda_model_m, corpus_m, id2word_m)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module i

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.391652 -0.039819       1        1  36.867729
14     0.387118 -0.166442       2        1  22.265289
10     0.352430 -0.208281       3        1  16.515009
18     0.257214  0.314915       4        1  10.038485
3      0.171079  0.174966       5        1   4.904688
0      0.069210  0.085849       6        1   3.228511
19     0.031511  0.100566       7        1   2.050869
1     -0.092428 -0.018660       8        1   0.813151
9     -0.101523 -0.019044       9        1   0.689778
4     -0.110085 -0.021087      10        1   0.550120
5     -0.119711 -0.021758      11        1   0.532654
2     -0.122752 -0.021839      12        1   0.422717
7     -0.129272 -0.021758      13        1   0.339806
15    -0.130331 -0.021697      14        1   0.256282
8     -0.137721 -0.020950      15        1   0.150457
6     -0.140355 -0.020218      16        1   0.146009
17    -0.141895 -0.019730      17        1   0.095158
12    -0.143864 -0.018877      18        1   0.075474
13    -0.144417 -0.018575      19        1   0.053902
16    -0.145862 -0.017562      20        1   0.003910, topic_info=               Term         Freq         Total Category  logprob  loglift
174         Franken  14911.00000  14911.000000  Default  30.0000  30.0000
176       Gemeinden  11456.00000  11456.000000  Default  29.0000  29.0000
209       Millionen  11792.00000  11792.000000  Default  28.0000  28.0000
133          Antrag  14039.00000  14039.000000  Default  27.0000  27.0000
107              ja  21208.00000  21208.000000  Default  26.0000  26.0000
..              ...          ...           ...      ...      ...      ...
24          konkret      0.00071    931.595080  Topic20 -11.9573  -3.9375
25            steht      0.00071   6035.379002  Topic20 -11.9573  -5.8060
26     unterstützen      0.00071  10961.524680  Topic20 -11.9573  -6.4027
27             voll      0.00071    423.615179  Topic20 -11.9573  -3.1494
29   übergeordneten      0.00071    174.562030  Topic20 -11.9573  -2.2629

[856 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3741      7  0.997164         Abgaben
2214      6  0.999541          Absatz
1947     17  0.986388          Ahnung
2981     15  0.976117            Alex
1152     14  0.946898       Amtsdauer
...     ...       ...             ...
404       7  0.999472        Änderung
405       7  0.999152      Änderungen
29        5  0.996780  übergeordneten
588       2  0.995159       überhaupt
588       3  0.004725       überhaupt

[677 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 15, 11, 19, 4, 1, 20, 2, 10, 5, 6, 3, 8, 16, 9, 7, 18, 13, 14, 17])

### Women

In [70]:
lda_viz = gensimvis.prepare(lda_model_w, corpus_w, id2word_w)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.453311  0.152971       1        1  64.175270
16    -0.360419  0.127951       2        1  12.789129
17    -0.154614 -0.237126       3        1   5.336812
6     -0.178108 -0.315255       4        1   5.243815
9      0.007533  0.001598       5        1   2.057662
13     0.012925 -0.003473       6        1   1.925252
11     0.035650  0.012270       7        1   1.626056
18     0.050197  0.016269       8        1   1.274136
4      0.049693  0.016090       9        1   1.181926
3      0.058713  0.018613      10        1   1.019246
12     0.081764  0.021537      11        1   0.728880
5      0.082646  0.021588      12        1   0.622910
1      0.087864  0.018546      13        1   0.447809
7      0.085034  0.023011      14        1   0.432610
15     0.098174  0.021366      15        1   0.315846
19     0.094570  0.021734      16        1   0.300637
8      0.093667  0.022030      17        1   0.222601
2      0.100665  0.020872      18        1   0.154224
0      0.102529  0.020300      19        1   0.121357
10     0.104829  0.019109      20        1   0.023820, topic_info=           Term         Freq        Total Category  logprob  loglift
640     Franken  5924.000000  5924.000000  Default  30.0000  30.0000
150   Gemeinden  5231.000000  5231.000000  Default  29.0000  29.0000
643   Millionen  4801.000000  4801.000000  Default  28.0000  28.0000
266     Prozent  4043.000000  4043.000000  Default  27.0000  27.0000
378       wurde  6495.000000  6495.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
6330      Strom     0.002857     0.979877  Topic20 -11.4426   2.5047
6311    Million     0.002857     0.979895  Topic20 -11.4426   2.5047
6536       KBIK     0.002857     0.980017  Topic20 -11.4426   2.5045
8433  Nachfrage     0.002857     0.979897  Topic20 -11.4426   2.5047
6212     vorher     0.002857     0.979895  Topic20 -11.4426   2.5047

[770 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
224       6  0.997030           1.
3186     10  0.996555          ABG
2885      8  0.990951  Abklärungen
1515      2  0.999251       Absatz
2675     19  0.833166       Abzüge
...     ...       ...          ...
4300      5  0.993321   überprüfen
955       6  0.997984   überweisen
1514      2  0.410650   überwiesen
1514      6  0.587127   überwiesen
743       6  0.994635   überzeugen

[565 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 17, 18, 7, 10, 14, 12, 19, 5, 4, 13, 6, 2, 8, 16, 20, 9, 3, 1, 11])